In [1]:
import os
from dataclasses import dataclass
import logging
logging.basicConfig(level=logging.INFO, 
    format='%(asctime)s %(levelname)s - %(message)s',
    datefmt="%H:%M:%S"
)

import numpy as np
%rm -rf my-experiment*
%rm -rf ioh_data*

In [2]:
import ioh
%pip show ioh

Name: ioh
Version: 0.3.2.8.3
Summary: The experimenter for Iterative Optimization Heuristics
Home-page: None
Author: Jacob de Nobel, Furong Ye, Diederick Vermetten, Hao Wang, Carola Doerr and Thomas Bäck
Author-email: iohprofiler@liacs.leidenuniv.nl
License: None
Location: /home/jacob/code/ioh-tutorial/venv/lib/python3.8/site-packages
Requires: numpy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


# Problem
In ioh, everything revolves around the `problem` class, which exists for both `Real` and `Integer` types for continious and discrete problems resp. These classes are wrappers around an objective function, and can be used to interact with the various other parts of iohexperimenter. 

We have a number of objective functions already implemented for convenience, which includes the objective functions from the BBOB single objective benchmark by the COCO platform for the continous case and the PBO benchmark functions for the discrete case.


In [3]:
# A list of problems can be accessed via the base classes
ioh.problem.Real.problems

{1: 'Sphere',
 2: 'Ellipsoid',
 3: 'Rastrigin',
 4: 'BuecheRastrigin',
 5: 'LinearSlope',
 6: 'AttractiveSector',
 7: 'StepEllipsoid',
 8: 'Rosenbrock',
 9: 'RosenbrockRotated',
 10: 'EllipsoidRotated',
 11: 'Discus',
 12: 'BentCigar',
 13: 'SharpRidge',
 14: 'DifferentPowers',
 15: 'RastriginRotated',
 16: 'Weierstrass',
 17: 'Schaffers10',
 18: 'Schaffers1000',
 19: 'GriewankRosenBrock',
 20: 'Schwefel',
 21: 'Gallagher101',
 22: 'Gallagher21',
 23: 'Katsuura',
 24: 'LunacekBiRastrigin'}

In [4]:
# In order to instantiate a problem instance, we can do the following:
problem = ioh.get_problem(
    "Sphere", 
    instance=1,
    dimension=10,
    problem_type="Real"
)
problem

<RealProblem 1. Sphere (iid=1 dim=10)>

In [5]:
# The problem class includes information about the problem, which can be retrieved via the meta_data accessor
problem.meta_data.name

'Sphere'

In [6]:
# The current state of the problem, e.g. the number of evaluations, best seen points etc. are stored in the problems state.
problem.state

<State evaluations: 0 optimum_found: false current_best: <Solution x: {nan, nan, nan, nan, nan, nan, nan, nan, nan, nan} y: nan>>

In [7]:
# Every problem as has a simple box-constraint associcated
problem.constraint

<Constraint lb: [{-5, -5, -5, -5, -5, -5, -5, -5, -5, -5}] ub: [{5, 5, 5, 5, 5, 5, 5, 5, 5, 5}]>

In [8]:
# We can access the contraint information of the problem
x0 = np.random.uniform(problem.constraint.lb, problem.constraint.ub)

# Evaluation happens like a 'normal' objective function would
problem(x0)

# Whenever the problem is evaluated, the state changes
problem.state

<State evaluations: 1 optimum_found: false current_best: <Solution x: {-1.2725000049293786, 1.6064946584641877, 2.2956320073145413, -4.567104048823011, -2.67266460951603, 3.1495602394346527, -1.4408130492208082, 2.202033314035824, 3.2453974567199477, -4.93745683250329} y: 181.56173085406095>>

In [9]:
# If we want to perform multiple runs with the same objective function, after every run, the problem has to be reset, 
# such that the internal state reflects the current run.
def run_experiment(problem, algorithm, n_runs=5):
    for run in range(n_runs):
        
        # Run the algorithm on the problem
        algorithm(problem)

        # print the best found for this run
        print(f"run: {run+1} - best found:{problem.state.current_best.y: .3f}")

        # Reset the problem
        problem.reset()

In [10]:
@dataclass
class RandomSearch:
    'Simple random search algorithm'
    
    n: int
    length: float = 0.0
        
    def __call__(self, problem: ioh.problem.Real) -> None:
        'Evaluate the problem n times with a randomly generated solution'
        
        for _ in range(self.n):
            # We can use the problems constraint accessor to get information about the problem bounds
            x = np.random.uniform(problem.constraint.lb, problem.constraint.ub)
            self.length = np.linalg.norm(x)
            
            problem(x)  
            

In [11]:
# using the random search algorithm, we can then run a simple experiment
run_experiment(problem, RandomSearch(10))

run: 1 - best found: 102.523
run: 2 - best found: 111.235
run: 3 - best found: 127.068
run: 4 - best found: 145.965
run: 5 - best found: 154.256


# Adding custom problems
As the list of already implemented problems might not contains the objective function you would like to analyze and benchmark, we include an easy to use interface to wrap your benchmark function into an iohexperimenter `problem` object. 

Currently, only single objective functions are supported, so the only requirement on the function is its signature, which should take a single array parameter, and return a single floating point number $f(\mathbf{x}) \mapsto \mathbb{R}$.

In this example we will add the function (Styblinski–Tang):

$f(\mathbf{x}) = \frac{\sum_{i=1}^n x_i^4 - 16x_i^2 + 5x_i}{2}$

defined on $[-5, 5]$

In [12]:
def styblinski_tang(x: np.ndarray) -> float:
    return np.sum(np.power(x, 4) - (16 * np.power(x, 2)) + (5 * x)) / 2


styblinski_tang(np.array([-2.903534]*10)) # global minima

-391.661657037714

In [13]:
# we can wrap this function in ioh, as this is is a continous function, we use wrap_real_problem:
ioh.problem.wrap_real_problem(
    styblinski_tang,                                     # Handle to the function
    name="StyblinskiTang",                               # Name to be used when instantiating
    optimization_type=ioh.OptimizationType.Minimization, # Specify that we want to minimize
    lb=-5,                                               # The lower bound
    ub=5,                                                # The upper bound
)

In [14]:
# We can create an instance of this problem, wrapped in ioh
problem = ioh.get_problem("StyblinskiTang", dimension=10)
problem

<RealProblem 25. StyblinskiTang (iid=1 dim=10)>

In [15]:
problem(np.array([-2.903534]*10))

-391.661657037714

## Specifying instances
When doing bechmarking of custom problems, it can often be usefull to look at different instances of the same problem, but that perform some transformation the parameter or objective space, to test if an algorithm is invariant to such transformations. 

Here we add the following transformations to the aforementioned objective function:

$T_x(\mathbf{x}, c) = \mathbf{x} + c$

$T_y(y, c) = y * c$

We can do this by providing transformation functions to the `wrap_problem` interface. Note that these take two parameters:
 1. The operand they function on, which are the variables for the variables transformation and objective function value for the objective transformation. 
 2. An integer identifier of the instance id which is currently used. This allows you to specify alternate behavior for different instances.  

In [16]:
# Variables transformation R^d -> R^d
def transform_variables(x: np.ndarray, instance_id:int) -> np.ndarray:
    c = (instance_id - 1) * .5
    return x + c

# Objective transformation R -> R
def transform_objectives(y: float, instance_id:int) -> float:
    c = instance_id
    return y * c

# Note that we can overwrite a previously defined problem by calling wrap_real_problem again with the same name
ioh.problem.wrap_real_problem(
    styblinski_tang,                                     
    name="StyblinskiTang",                               
    optimization_type=ioh.OptimizationType.Minimization, 
    lb=-5,                                               
    ub=5,      
    
    # Adding the transformation functions
    transform_variables=transform_variables,     
    transform_objectives=transform_objectives
)

In [17]:
# We can now create different instances of the same problem
instance1 = ioh.get_problem("StyblinskiTang", instance=1, dimension=10)
instance2 = ioh.get_problem("StyblinskiTang", instance=2, dimension=10)
instance1, instance2

(<RealProblem 25. StyblinskiTang (iid=1 dim=10)>,
 <RealProblem 25. StyblinskiTang (iid=2 dim=10)>)

In [18]:
# Note that when evaluating with the same point, each instance gives a different (transformed value)
instance1(x0), instance2(x0)

(-42.48557436387017, -223.2909736182682)

# Logging data
The default usage of IOHExperimenter is in generating logs of benchmarking experiments which can be analyzed in IOHAnalyzer. 

In [19]:
import os

logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="my-experiment",       # in a folder named: 'my-experiment'
    algorithm_name="random-search",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

# this automatically creates a folder 'my-experiment' in the current working directory
# if the folder already exists, it will given an additional number to make the name unique
logger

<Analyzer /home/jacob/code/ioh-tutorial/my-experiment>

In [20]:
%ls

'IOHExperimenter Tutorial.ipynb'   my-experiment/   requirements.txt   venv/


In [21]:
# In order to log data for a problem, we only have to attach it to a logger
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(logger)

# We can then run the random search as before, only now all data will be logged to a file
run_experiment(problem, RandomSearch(10), n_runs=1)

run: 1 - best found:-73.110


In [22]:
%cat my-experiment/IOHprofiler_f25_StyblinskiTang.info

suite = "unknown_suite", funcId = 25, funcName = "StyblinskiTang", DIM = 2, maximization = "F", algId = "random-search", algInfo = "algorithm_info"
%
data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat, 1:5|-73.1097

In [23]:
%cat my-experiment/data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat

"function evaluation" "current f(x)" "best-so-far f(x)" "current af(x)+b" "best af(x)+b" x0 x1
1 65.5936859531 65.5936859531 65.5936859531 65.5936859531 2.158278 4.755001
2 -28.4974746912 -28.4974746912 -28.4974746912 -28.4974746912 -0.527632 2.694358
5 -73.1097338138 -73.1097338138 -73.1097338138 -73.1097338138 -2.384464 -3.169940
10 -43.8233617411 -73.1097338138 -43.8233617411 -73.1097338138 -1.616690 2.271519


## Triggers 
The default behavior of the `Analyzer` logger is to log data only when there is an improvement of the objective value. We can change this behaviour, by specifying one or more triggers, which are logical operators, which when one of them evaluates to True, will cause data to be logged.

We provide a number of trigger variants which can be use to customize the logging. In the following example, a trigger is defined which evaluates to True, every 3 function evaluations. It is combined with a trigger for improvement, so data will be logged on every 3rd function evaluation, or when there is an observed improvement of the objective value

In [24]:
triggers = [
    ioh.logger.trigger.Each(3),
    ioh.logger.trigger.OnImprovement()
]

logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="my-experiment",       
    algorithm_name="random-search",    
    store_positions=True,
    
    # Add the triggers to the logger
    triggers = triggers
)

logger

<Analyzer /home/jacob/code/ioh-tutorial/my-experiment-1>

In [25]:
# Rerun the same experiment as before
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(logger)
run_experiment(problem, RandomSearch(10), n_runs=1)

run: 1 - best found:-41.013


In [26]:
# We can now see that data is logged either if there is improvement, or on every 3rd evaluation
%cat my-experiment-1/data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat

"function evaluation" "current f(x)" "best-so-far f(x)" "current af(x)+b" "best af(x)+b" x0 x1
1 49.1047609060 49.1047609060 49.1047609060 49.1047609060 3.827346 4.456120
3 -41.0132926148 -41.0132926148 -41.0132926148 -41.0132926148 -0.816301 -3.413567
4 -29.2326195328 -41.0132926148 -29.2326195328 -41.0132926148 -3.584684 0.326907
6 -35.3549105247 -41.0132926148 -35.3549105247 -41.0132926148 -3.823978 1.769667
9 97.8908537588 -41.0132926148 97.8908537588 -41.0132926148 4.997429 -3.658043
10 -35.1711956989 -41.0132926148 -35.1711956989 -41.0132926148 3.400643 -3.856496


## Properties
If we want to keep track of any dynamic parameters a given algorithm might have, we can use properties to log them to the output files. 

In the following example, we will track the length parameters for the RandomSearch algorithm, which is added for illustrative purpoposes, and changes for every function evaluation

In [27]:
# RandomSearch has a length parameter, which is dynamic
algorithm = RandomSearch(10)

# Creating a new logger
logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="my-experiment",       
    algorithm_name="random-search",    
    store_positions=True
)

# Before we attach a problem, we tell the logger to keep track of the length parameter on algorithm
logger.watch(algorithm, "length")

# We can now again run the same experiment 
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)

problem.attach_logger(logger)
run_experiment(problem, algorithm, n_runs=1)

run: 1 - best found:-46.375


In [28]:
# Note the additional length parameter being logged 
%cat my-experiment-2/data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat

"function evaluation" "current f(x)" "best-so-far f(x)" "current af(x)+b" "best af(x)+b" length x0 x1
1 10.1485546897 10.1485546897 10.1485546897 10.1485546897 4.5278702302 -1.536037 4.259366
2 0.5318005976 0.5318005976 0.5318005976 0.5318005976 4.0144774288 3.911949 0.901489
3 -35.7448641179 -35.7448641179 -35.7448641179 -35.7448641179 2.3402961605 -0.279169 -2.323586
10 -46.3746692964 -46.3746692964 -46.3746692964 -46.3746692964 2.4060206689 -1.770710 -1.628964


# Alternate logging behaviour
We provide a number of different loggers in addition to the `Analyzer` logger, which include:
 - `FlatFile` which logs data to a simple csv file
 - `Store` which keeps all of the stored data in memory
 - `EAF/EAH` which compute Empirical Attainment Function/Histogram statistics on the fly

You can define your own custom logging behavoir by inheriting from the `AbstractLogger` class. The only required part of the interface is that you override the `__call__` operator, which takes a single `ioh.LogInfo` parameter. In this method you should define your desired behavior. 

In [29]:
# Simple logger that logs data using the python logging module whenever it is triggeredd
class MyLogger(ioh.logger.AbstractLogger):
    def __call__(self, log_info: ioh.LogInfo):
        logging.info(msg=f"triggered! y: {log_info.current.y}")


# The abstract logger takes two parameters, triggers and properties
mylogger = MyLogger(triggers=[ioh.logger.trigger.ALWAYS])

problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(mylogger)

run_experiment(problem, RandomSearch(10), 1)

11:33:30 INFO - triggered! y: -10.197419838378643
11:33:30 INFO - triggered! y: -42.79813100825032
11:33:30 INFO - triggered! y: -0.29719556658903135
11:33:30 INFO - triggered! y: -39.01173136188547
11:33:30 INFO - triggered! y: -4.7352531474134905
11:33:30 INFO - triggered! y: -31.27505992893284
11:33:30 INFO - triggered! y: 2.7197161132209944
11:33:30 INFO - triggered! y: -44.04618375686107
11:33:30 INFO - triggered! y: -37.25753140377591
11:33:30 INFO - triggered! y: -1.0302052630371803


run: 1 - best found:-44.046


# Standardized Experimental Setup (Python only)
In Python, we provide the `Experiment` class which can be used to easily run a given algorithm over a larger number of problems. 

In [30]:
experiment = ioh.Experiment(
    algorithm = RandomSearch(10), # An algorithm instance
    fids = [1, 25],               # the id's of the problems we want to test
    iids = [1, 10],               # the instances 
    dims = [2, 10],               # the dimensions
    reps = 3,                     # the number of runs,
    zip_output = True       
)

In [31]:
experiment.run()

In [34]:
%ls ioh_data

IOHprofiler_f1_Sphere.info           data_f1_Sphere/
IOHprofiler_f25_StyblinskiTang.info  data_f25_StyblinskiTang/


In [35]:
%cat ioh_data/IOHprofiler_f25_StyblinskiTang.info

suite = "unknown_suite", funcId = 25, funcName = "StyblinskiTang", DIM = 10, maximization = "F", algId = "RandomSearch(n=10, length=0.0)", algInfo = "algorithm_info"
%
data_f25_StyblinskiTang/IOHprofiler_f25_DIM10.dat, 10:6|25370.7, 10:6|20553.4, 10:8|9818.67
suite = "unknown_suite", funcId = 25, funcName = "StyblinskiTang", DIM = 10, maximization = "F", algId = "RandomSearch(n=10, length=0.0)", algInfo = "algorithm_info"
%
data_f25_StyblinskiTang/IOHprofiler_f25_DIM10.dat, 1:4|-160.946, 1:3|-148.304, 1:6|-203.145
suite = "unknown_suite", funcId = 25, funcName = "StyblinskiTang", DIM = 2, maximization = "F", algId = "RandomSearch(n=10, length=0.0)", algInfo = "algorithm_info"
%
data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat, 1:2|-68.4465, 1:2|-68.02, 1:6|-59.8526
suite = "unknown_suite", funcId = 25, funcName = "StyblinskiTang", DIM = 2, maximization = "F", algId = "RandomSearch(n=10, length=0.0)", algInfo = "algorithm_info"
%
data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat, 10:1|-3

In [38]:
%ls 

'IOHExperimenter Tutorial.ipynb'   my-experiment/     requirements.txt
 ioh_data/                         my-experiment-1/   venv/
 ioh_data.zip                      my-experiment-2/
